# Best agent

In [ ]:
%%capture
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets",configuration={"randomSeed":42}, debug=True)
# env.run(["../input/kore-mining-labo-note-v3/main3.py"])
# env.render(mode="ipython", width=700, height=700)

In [ ]:
import sys
sys.path.append("../input/kore-pixyz-library")
from steps_analyze import steps_analyze
from steps_compare import steps_compare
from do_nothing import do_nothing

#steps_analyze(steps)

pixyz  
last update 2022 06 18  
ゆっくりしていってね！  

**霊夢:今回はコンペ理解のために、Miningについて検証していくぞ。**

**魔理沙:他に気になったことがあったら突っ込んでいくぞ。**

**Reimu: This time, I'll examine Mining to understand the competition.**

**Marisa: If you have any other concerns, I'll dig in.**

<img src="https://4.bp.blogspot.com/-uoVuBWIbdiA/WvQHqpx_YCI/AAAAAAABL8g/NiFZ6K71VBc_0_dcKb3_4nhnvFJ_JMNuACLcBGAs/s450/network_dennou_sekai_figure.png" width = 300>

# Contents
<span style="font-size: 200%; color: black;">
    
* v1 [**number of box_mining**](#number-of-box-mining)

* v2 [**Size comparison**](#v2-Size-comparison)

* v3 [**Use observation**](#Use-observation)

* v4 [**refactering **](#v4-refactering)

</span>

# Method

**霊夢:色々なagentを、特定のマップで動かしてみて、以下の4つの変動をグラフにして比較してみるよ。**

* Kore:Shipyardsが持っている資源の数
* Cargo:艦隊が持っている資源の数
* Ships:全体が持っている船の数
* All:これまでに獲得した資源の数 (Kore + Cargo + Ships×10 - 500)

**10,500 の値はそれぞれ、造船のコスト、ゲーム開始時に所持している資源の数の事です。**

**魔理沙:randomSeed = 42に設定して、Koreの配置は全検証において同じになるようにしたぜ**

**Reimu: Try running different agents on a specific map and compare the following four fluctuations in a graph.**

* Kore: Number of resources that Shipyards have
* Cargo: The number of resources the fleet has
* Ships: Number of ships the player has
* All: Number of resources acquired so far (Kore + Cargo + Ships × 10 -500)

**Values of 10,500 are the cost of shipbuilding and the number of resources you have at the start of the game, respectively.**

**Marisa: I set randomSeed = 42 so that Kore's placement is the same for all validations**

<img src="https://3.bp.blogspot.com/-eZg7ny24Qkk/WkR91IX_MdI/AAAAAAABJVU/_E8oVVwiaRwUj2SN9h8hsIKpMR0yt3WgQCLcBGAs/s400/kabu_chart_man.png" width = 200>

In [ ]:
import matplotlib.pylab as plt
# from kaggle_environments import make

# env = make("kore_fleets",configuration={"randomSeed":42}, debug=True)
# print(env.name, env.version)

# v1 number of box_mining

## Code

In [ ]:
%%writefile mining_test.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

##-----------functions--------------
def one_box_mining(agent,turn):
    period = 40
    ship_count = [21] + [3]*8 + [2]*2
    flight_plan = ["E9N9W9S"] +["E"+str(8-x)+"N" for x in range(8)] + ["EN","N"]
    for shipyard in agent.shipyards:
        action = None
        if turn < 40:
            action = ShipyardAction.spawn_ships(1)
        elif turn % period < 22 and turn % 2:
            i = (turn % period - 1) // 2
            action = ShipyardAction.launch_fleet_with_flight_plan(ship_count[i], flight_plan[i])            
        shipyard.next_action = action
        
    return

def two_box_mining(agent,turn,param):
    box_size = 10
    period = 40
    r = [10,30]
    for k,shipyard in enumerate(agent.shipyards):
        if not shipyard.next_action == None:
            continue
        if shipyard.ship_count >= 21 and turn % period == r[0] and param.box_num[k] < 1:
            param.box_num[k] = 1
        if shipyard.ship_count >= 68 and turn % period == r[1] and param.box_num[k] < 2:
            param.box_num[k] = 2
        action = None
        
        Dir = ['ENWS','WSEN']
        for num in range(param.box_num[k]):
            for i in range(box_size):
                if turn % period == (2*i + r[num]) % period:
                    if i == 0:
                        flight_plan = ''
                        for j in range(3):
                            flight_plan += Dir[num][j] + str(box_size-1)
                        flight_plan += Dir[num][3]
                        action = ShipyardAction.launch_fleet_with_flight_plan(21, flight_plan)            
                    elif i == box_size:
                        flight_plan = Dir[num][1]
                        action = ShipyardAction.launch_fleet_with_flight_plan(2, flight_plan)            
                    elif i == box_size - 1:
                        flight_plan = Dir[num][:2]
                        action = ShipyardAction.launch_fleet_with_flight_plan(2, flight_plan)            
                    else:
                        flight_plan = Dir[num][0] + str(box_size - i - 1) + Dir[num][1]
                        action = ShipyardAction.launch_fleet_with_flight_plan(3, flight_plan)            
            
        shipyard.next_action = action
        
    return

def four_box_mining(agent,turn,param):
    box_size = 10
    period = 40
    r = [10,11,30,31]
    for k,shipyard in enumerate(agent.shipyards):
        if not shipyard.next_action == None:
            continue
        if shipyard.ship_count >= 21 and turn % period == r[0] and param.box_num[k] < 1:
            param.box_num[k] = 1
        if shipyard.ship_count >= 94 and turn % period == r[1] and param.box_num[k] < 2:
            param.box_num[k] = 2
        if shipyard.ship_count >= 141 and turn % period == r[2] and param.box_num[k] < 3:
            param.box_num[k] = 3
        if shipyard.ship_count >= 188 and turn % period == r[3] and param.box_num[k] < 4:
            param.box_num[k] = 4
            
        action = None
        
        Dir = ['ENWS','WSEN','ESWN','WNES']

        for num in range(param.box_num[k]):
            for i in range(box_size):
                if turn % period == (2*i + r[num]) % period:
                    if i == 0:
                        flight_plan = ''
                        for j in range(3):
                            flight_plan += Dir[num][j] + str(box_size-1)
                        flight_plan += Dir[num][3]
                        action = ShipyardAction.launch_fleet_with_flight_plan(21, flight_plan)            
                    elif i == box_size:
                        flight_plan = Dir[num][1]
                        action = ShipyardAction.launch_fleet_with_flight_plan(2, flight_plan)            
                    elif i == box_size - 1:
                        flight_plan = Dir[num][:2]
                        action = ShipyardAction.launch_fleet_with_flight_plan(2, flight_plan)            
                    else:
                        flight_plan = Dir[num][0] + str(box_size - i - 1) + Dir[num][1]
                        action = ShipyardAction.launch_fleet_with_flight_plan(3, flight_plan)            
            
        shipyard.next_action = action
        
    return

def spawn(agent,kore_left,spawn_cost):
    for shipyard in agent.shipyards:
        if not shipyard.next_action == None:
            continue
        if kore_left >= spawn_cost:
            shipyard.next_action = ShipyardAction.spawn_ships(1)
    return

def max_spawn(agent,spawn_cost,kore_left,param):
    for i,shipyard in enumerate(agent.shipyards):
        if shipyard.next_action:
            continue
        if kore_left >= spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            shipyard.next_action = action
            kore_left -= spawn_cost * shipyard.max_spawn
            param.ship_num[i] += shipyard.max_spawn
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
            kore_left -= spawn_cost
            param.ship_num[i] += 1
    return

##-----------classes-------------
class param:
    box_num = [0]
    ship_num = [0]
    shipyard_count = 1
    max_shipyard = 1

##-----------agents--------------
def one_box_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    
    one_box_mining(me,turn)
    spawn(me,kore_left,spawn_cost)
    
    return me.next_actions

def two_box_agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    two_box_mining(me,turn,param)
    max_spawn(me,spawn_cost,kore_left,param)
    return me.next_actions

def four_box_agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    four_box_mining(me,turn,param)
    max_spawn(me,spawn_cost,kore_left,param)
    return me.next_actions

In [ ]:
%%writefile make_ship_old.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def spawn(agent,kore_left,spawn_cost):
    for shipyard in agent.shipyards:
        if not shipyard.next_action == None:
            continue
        if kore_left >= spawn_cost:
            shipyard.next_action = ShipyardAction.spawn_ships(1)
    return

def max_spawn(agent,spawn_cost,kore_left,param):
    for i,shipyard in enumerate(agent.shipyards):
        if shipyard.next_action:
            continue
        if kore_left >= spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            shipyard.next_action = action
            kore_left -= spawn_cost * shipyard.max_spawn
            param.ship_num[i] += shipyard.max_spawn
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
            kore_left -= spawn_cost
            param.ship_num[i] += 1
    return

## 1-1 one_box_mining

**魔理沙:Kore Intro IIに出てきたbox_minerだぜ。**

**Marisa: It's the box_miner from Kore Intro II.**

In [ ]:
from mining_test import one_box_agent
step1 = env.run([one_box_agent])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step1)

## 1-2 two_box_mining

In [ ]:
from mining_test import two_box_agent
step2 = env.run([two_box_agent])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step2)

## 1-3 four_box_miner

In [ ]:
from mining import four_box_agent
step3 = env.run([four_box_agent])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step3)

## 1-4 one_box_mining vs two_box_mining vs four_box_mining

In [ ]:
step_list = [step1,step2,step3]
agent_name = ["one_box_mining","two_box_mining","four_box_mining"]
steps_compare(step_list,agent_name)

**魔理沙:Koreは、200ターン目までどのエージェントも同じくらいだけど、最終的には、four_box_miningが一番多くなってるぜ。**

**霊夢:Cargoは150ターン目から増加してきて、300ターン目あたりからは振幅がほぼ一定になるね。**

**魔理沙:Shipsは200ターン目あたりまではどのエージェントも同じくらいだけど、そこからone_box_miningは少しずつ増加していって、two_box_miningは大きく増加していって、four_box_miningは完全に増加しなくなるね。four_box_miningが増加しなくなるのは、艦隊を出し続けているせいで、造船する暇がないからだろうな**

**霊夢:All (取得した資源の総数：Kore + Cargo + Ships*10 -500) は、150ターン目あたりまではどのエージェントも大差ないけど、そこから差が開いて行って、最終的にはfour_box_miningが最も多くなったよ。**

**魔理沙:two_box_miningはone_box_miningの約2倍になってるけど、for_box_miningはtwo_box_miningの2倍やone_box_miningの4倍より少なそうだね。**

**Marisa: Kore is about the same for all agents until turn 200, but in the end, four_box_mining is the most.**

**Reimu: Cargo increases from turn 150, and the amplitude becomes almost constant from around turn 300.**

**Marisa: Ships is about the same for all agents until around turn 200, but from there one_box_mining is increasing little by little, two_box_mining is increasing significantly, and four_box_mining isn't increasing altogether. The reason why four_box_mining doesn't increase is probably because we don't have time to build a ship because we keep out the fleet**

**Reimu: All (total number of resources acquired: Kore + Cargo + Ships * 10 -500) is not much different for any agent until around the 150th turn, but the difference comes out from there, and finally four_box_mining is the most.**

**Marisa: Two_box_mining is about twice as much as one_box_mining, but for_box_mining is less than twice as much as two_box_mining and four times as much as one_box_mining.**

## 1-5 create_flight_course,kore_cfg,make_ship

In [ ]:
%%writefile create_flight_course_test.py

def create_flight_course_square(ew,ns):
    s_dir='ESWN'
    s_ew=str(ew-1) if ew>1 else s_ew=str(-ew-1) if ew<-1 else '' 
    s_ns=str(ns-1) if ns>1 else s_ns=str(-ns-1) if ns<-1 else '' 
    if ew<0: s_dir[0],s_dir[2]=s_dir[2],s_dir[0]
    if ns<0: s_dir[1],s_dir[3]=s_dir[1],s_dir[3]
        
    return s_dir[0]+s_ew+s_dir[1]+s_ns+s_dir[2]+s_ew+s_dir[3]

def create_flight_course_L(ew,ns):
    s_dir='ESWN'
    s_ew=str(ew-1) if ew>1 else s_ew=str(-ew-1) if ew<-1 else '' 
    if ew<0: s_dir[0],s_dir[2]=s_dir[2],s_dir[0]
    if ns<0: s_dir[1],s_dir[3]=s_dir[1],s_dir[3]
 
   return s_dir[0]+s_ew+s_dir[1]


def create_flight_course_box(ew,ns):
    res=[create_flight_course_square(ew,ns)]
    if ew>0:
        res=res+[create_flight_course_L(ew-x-1,ns) for x in range(ew-1) ]
    else:
        res=res+[create_flight_course_L(ew+x+1,ns) for x in range(-ew-1) ]
    return res    



In [ ]:
%%writefile kore_cfg_test.py

class CFG:
    spawn_cost=10

In [ ]:
%%writefile make_ship_test.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
from kore_cfg_test import CFG

def spawn_all(agent):
    kore_left=agent.kore
    for shipyard in agent.shipyards:
        if not shipyard.next_action == None:
            continue
        n_ship=kore_left//CFG.spawn_cost
        if n_ship>shipyard.max_spawn:
            n_ship=shipyard.max_spawn
        if n_ship >0:
            shipyard.next_action = ShipyardAction.spawn_ships(n_ship)
            kore_left-=CFG.spawn_cost*n_ship
    return

def spawn(agent,shipyard,remain_kore):
    n_ship=remain_kore//CFG.spawn_cost
    if n_ship>shipyard.max_spawn:
        n_ship=shipyard.max_spawn
    if n_ship >0:
        shipyard.next_action = ShipyardAction.spawn_ships(n_ship)
        remain_kore-=CFG.spawn_cost*n_ship
    return

処理をagent毎から、shipyard毎に変えます。また、flight_planを辞書型にします。

In [ ]:
%%writefile mining_test2.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
from make_ship import spawn,spawn_all
from create_flight_course import create_flight_course_box

##-----------functions--------------
def box_mining(agent,shipyard,turn,flight_plan,ew=5,ns=5):
    min_fleets=21+3*(ew-1)
    ship_count=shipyard.ship_count-reserve_ship(shipyard.id,turn)
    if ship_count >=min_fleets:
        course=create_flight_course_box(ew,ns)
        main_ship_count=max(21,ship_count//abs(ew))
        sub_ship_count=min((ship_count-main_ship_count)//(ew-1), main_ship_count-1)
        for i in range(ew):
            flight_plan[shipyard.id]['n_fleets'][turn+i*2]=n_ships[i] 
            flight_plan[shipyard.id]['course'][turn+i*2]=course[i] 
        shipyard.next_action=ShipyardAction.launch_fleet_with_flight_plan(n_ships[0], course[0])        
    return
##-----------classes-------------
flight_plan={'next_kore':[0]*441}
remain_kore=0
def _create_flight_plan(shipyard_id):     
    flight_plan[shipyard_id]={}
    flight_plan[shipyard_id]['n_fleets']=[0]*480
    flight_plan[shipyard_id]['course']=[""]*480

def reserve_ship(shipyard_id,turn):
    return sum(flight_plan[shipyard_id]['n_fleets'][turn:])
   
    
##-----------agents--------------
def one_box_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    turn = board.step
    remain_kore=me.kore
    for shipyard in me.shipyards:
        if shipyard.id not in flight_plan:
            _create_flight_plan(shipyard.id
        n_fleets=flight_plan[shipyard.id]['n_fleets'][turn]
        if n_fleets>0:
            flight_course=flight_plan[shipyard.id]['course'][turn]
            shipyard.next_action=ShipyardAction.launch_fleet_with_flight_plan(n_fleets, flight_course)
        else;
            one_box_mining(me,shipyard,turn,flight_plan,ew=10,ns=10)
        if shipyard.next_action == None:
            spawn(me,shipyard,remain_kore)
    
    return me.next_actions

def two_box_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    turn = board.step
    remain_kore=me.kore
    for shipyard in me.shipyards:
        if shipyard.id not in flight_plan:
            _create_flight_plan(shipyard.id
        n_fleets=flight_plan[shipyard.id]['n_fleets'][turn]
        if n_fleets>0:
            flight_course=flight_plan[shipyard.id]['course'][turn]
            shipyard.next_action=ShipyardAction.launch_fleet_with_flight_plan(n_fleets, flight_course)
        else;
            if (turn% 2)>0:
                one_box_mining(agent,shipyard,turn,flight_plan,ew=10,ns=10):
            else:
                one_box_mining(agent,shipyard,turn,flight_plan,ew=-10,ns=-10):
        if shipyard.next_action == None:
            spawn(me,shipyard,remain_kore)
    
    return me.next_actions

def four_box_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    turn = board.step
    remain_kore=me.kore
    for shipyard in me.shipyards:
        if shipyard.id not in flight_plan:
            _create_flight_plan(shipyard.id
        n_fleets=flight_plan[shipyard.id]['n_fleets'][turn]
        if n_fleets>0:
            flight_course=flight_plan[shipyard.id]['course'][turn]
            shipyard.next_action=ShipyardAction.launch_fleet_with_flight_plan(n_fleets, flight_course)
        else;
            turn_side=turn% 4
            if turn_side ==0:
                one_box_mining(agent,shipyard,turn,flight_plan,ew=5,ns=5):
            if turn_side ==1:
                one_box_mining(agent,shipyard,turn,flight_plan,ew=5,ns=-5):
            if turn_side ==2:
                one_box_mining(agent,shipyard,turn,flight_plan,ew=-5,ns=-5):
            if turn_side ==3:
                one_box_mining(agent,shipyard,turn,flight_plan,ew=-5,ns=5):
        if shipyard.next_action == None:
            spawn(me,shipyard,remain_kore)
    
    return me.next_actions


# v2 Size comparison

## Code

In [ ]:
%%writefile mining_test_plus.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
from make_ship_old import max_spawn

##-----------functions-------------
def four_box_minning_size(agent,turn,param,box_size = 8):
    period = box_size * 4
    r = [10,11,(10 + box_size * 2)%period,(11 + box_size * 2)%period]
    for k,shipyard in enumerate(agent.shipyards):
        if not shipyard.next_action == None:
            continue
        if shipyard.ship_count >= 21 and turn % period == r[0] and param.box_num[k] < 1:
            param.box_num[k] = 1
        if shipyard.ship_count >= (17+3*box_size)*2 and turn % period == r[1] and param.box_num[k] < 2:
            param.box_num[k] = 2
        if shipyard.ship_count >= (17+3*box_size)*3 and turn % period == r[2] and param.box_num[k] < 3:
            param.box_num[k] = 3
        if shipyard.ship_count >= (17+3*box_size)*4 and turn % period == r[3] and param.box_num[k] < 4:
            param.box_num[k] = 4
            
        action = None
        
        Dir = ['ENWS','WSEN','ESWN','WNES']

#         add_ship = 0 if param.box_num[k] == 0 else (param.ship_num[k] - 32 * param.box_num[k])//(param.box_num[k]*box_size)
#         if param.shipyard_count < param.max_shipyard : add_ship = min(add_ship,15)
#         add_ship = max(add_ship,0)
        add_ship = 0
        for num in range(param.box_num[k]):
            for i in range(box_size):
                if turn % period == (2*i + r[num]) % period:
                    if i == 0:
                        flight_plan = ''
                        for j in range(3):
                            flight_plan += Dir[num][j] + str(box_size-1)
                        flight_plan += Dir[num][3]
                        action = ShipyardAction.launch_fleet_with_flight_plan(21 + add_ship, flight_plan)            
                    elif i == box_size:
                        flight_plan = Dir[num][1]
                        action = ShipyardAction.launch_fleet_with_flight_plan(2 + add_ship, flight_plan)            
                    elif i == box_size - 1:
                        flight_plan = Dir[num][:2]
                        action = ShipyardAction.launch_fleet_with_flight_plan(2 + add_ship, flight_plan)            
                    else:
                        flight_plan = Dir[num][0] + str(box_size - i - 1) + Dir[num][1]
                        action = ShipyardAction.launch_fleet_with_flight_plan(3 + add_ship, flight_plan)            
            
        shipyard.next_action = action
        
    return

def param_reset(param):
    param.box_num = [0]
    param.ship_num = [0]
    param.shipyard_count = 1
    param.max_shipyard = 1
    return
    
##-----------classes-------------
class param:
    box_num = [0]
    ship_num = [0]
    shipyard_count = 1
    max_shipyard = 1

##-----------agents-------------
def size8_agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    turn = board.step
    if turn == 0: param_reset(param)
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    four_box_minning_size(me,turn,param)
    max_spawn(me,spawn_cost,kore_left,param)
    return me.next_actions

def size5_agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    turn = board.step
    if turn == 0: param_reset(param)
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    four_box_minning_size(me,turn,param,box_size = 5)
    max_spawn(me,spawn_cost,kore_left,param)
    return me.next_actions


# 2-1 four_mining(size = 8)

In [ ]:
from mining_test_plus import size8_agent
step5 = env.run([size8_agent])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step5)

# 2-2 four_mining(size = 5)

In [ ]:
from mining_test_plus import size5_agent
step6 = env.run([size5_agent])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step6)

# 2-3 10size(1-3) vs 8size(2-1) vs 5size(2-2)

In [ ]:
step_list = [step3,step5,step6]
agent_name = ["10size(1-3)","8size(2-1)","5size(2-2)"]
steps_compare(step_list,agent_name)

**霊夢:Koreは150～250ターン目にかけて、5sizeが一番多くなっているね**

**魔理沙:Cargoは、サイズが大きくなるほど多くなってるね。**

**霊夢:Shipは、サイズが小さいほど序盤の生産が早いね。**

**魔理沙:Allは、序盤はサイズが小さいほど高いが、終盤はサイズが大きい方が高くなっているね。**

**Reimu: Kore has the most 5sizes from turn 150 to 250.**

**Marisa: Cargo gets bigger as it gets bigger.**

**Reimu: The smaller the size of Ship, the faster it will be produced in the early stages.**

**Marisa: All is higher in the early stages as the size is smaller, but in the final stages it is higher as the size is larger.**

# v3 Use observation

**魔理沙:observationを使って、より資源の多い航路を選ぶagentを作ってみるぜ。**

## Code

In [ ]:
%%writefile mining_useobs_test.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
import numpy as np
import re
from make_ship_old import max_spawn

##-----------functions-------------
def box_mining_bestdir(obs,config,agent,turn,param,box_size = 5,same_dir = True):
    period = box_size * 4
    r = [10,11,(10 + box_size*2)%period,(11 + box_size*2)%period]
    for k,shipyard in enumerate(agent.shipyards):
        if not shipyard.next_action == None:
            continue
        if param.ship_num[k] >= 21 and turn % period == r[0] and param.box_num[k] < 1:
            param.box_num[k] = 1
        if param.ship_num[k] >= 64 and turn % period == r[1] and param.box_num[k] < 2:
            param.box_num[k] = 2
        if param.ship_num[k] >= 96 and turn % period == r[2] and param.box_num[k] < 3:
            param.box_num[k] = 3
        if param.ship_num[k] >= 128 and turn % period == r[3] and param.box_num[k] < 4:
            param.box_num[k] = 4
            
        action = None

        for num in range(param.box_num[k]):
            if turn % period == r[num]:
                param.DIR[num] = select_dir(obs,shipyard,box_size,turn,num,param,same_dir)
        
        Dir = param.DIR
#         add_ship = 0 if param.box_num[k] == 0 else (param.ship_num[k] - 32 * param.box_num[k])//(param.box_num[k]*box_size)
#         add_ship = max(add_ship,0)
        add_ship = 0
        for num in range(param.box_num[k]):
            for i in range(box_size):
                if turn % period == (2*i + r[num]) % period:
                    if i == 0:
                        flight_plan = ''
                        for j in range(3):
                            flight_plan += Dir[num][j] + str(box_size-1)
                        flight_plan += Dir[num][3]
                        action = ShipyardAction.launch_fleet_with_flight_plan(21 + add_ship, flight_plan)            
                    elif i == box_size:
                        flight_plan = Dir[num][1]
                        action = ShipyardAction.launch_fleet_with_flight_plan(2 + add_ship, flight_plan)            
                    elif i == box_size - 1:
                        flight_plan = Dir[num][:2]
                        action = ShipyardAction.launch_fleet_with_flight_plan(2 + add_ship, flight_plan)            
                    else:
                        flight_plan = Dir[num][0] + str(box_size - i - 1) + Dir[num][1]
                        action = ShipyardAction.launch_fleet_with_flight_plan(3 + add_ship, flight_plan)            
            
        shipyard.next_action = action
        
    return

def select_dir(obs,shipyard,box_size,turn,num,param,same_dir):
    Dir = ['ENWS','WSEN','ESWN','WNES']
    x = shipyard.position.x
    y = 20 - shipyard.position.y
    kore = obs['kore']
    kore = np.array(kore).reshape((21,21))
    kore = np.roll(kore,(10-y,10-x),axis = (0,1))
    kx = [10,10-box_size,10,10-box_size]
    ky = [10-box_size,10,10,10-box_size]
    best = ['ENWS',0]
#     print(turn)
#     print(x,y)
    for i,DIR in enumerate(Dir):
        score = kore[ky[i]:ky[i]+box_size+1,kx[i]:kx[i]+box_size+1].sum()
        if num % 2 and Dir[i] == param.DIR[num-1] and not same_dir:
            score *= 0.9
        if score > best[1]:
            best[0] = Dir[i]
            best[1] = score
#         print(kore[ky[i]:ky[i]+box_size+1,kx[i]:kx[i]+box_size+1])
#         print(Dir[i],score)
        
    return best[0]

def bestcorse_minning(obs,config,agent,turn,param,same_corse = True):
    for k,shipyard in enumerate(agent.shipyards):
        if not shipyard.next_action == None:
            continue
        if shipyard.ship_count < 21:
            continue
            
        action = None
        
        box_size_x,box_size_y,Dir = select_corse(obs,shipyard,turn,param,same_corse)
        
        DIR = ['ENWS','WSEN','ESWN','WNES']
        kx = [10,10 - box_size_x,10,10 - box_size_x]
        ky = [10 - box_size_y,10,10,10 - box_size_y]
        idx = DIR.index(Dir)
        param.kore_his[ky[idx]:ky[idx]+box_size_y+1,kx[idx]:kx[idx]+box_size_x+1] += 5
        param.kore_his[ky[idx]+1:ky[idx]+box_size_y,kx[idx]+1:kx[idx]+box_size_x] -= 5

#         add_ship = 0 if param.box_num[k] == 0 else (param.ship_num[k] - 32 * param.box_num[k])//(param.box_num[k]*box_size)
#         add_ship = max(add_ship,0)
        add_ship = 0
        flight_plan = ''
        flight_plan += Dir[0] + str(box_size_x-1)
        flight_plan += Dir[1] + str(box_size_y-1)
        flight_plan += Dir[2] + str(box_size_x-1)                        
        flight_plan += Dir[3]
        action = ShipyardAction.launch_fleet_with_flight_plan(21 + add_ship, flight_plan)            

        shipyard.next_action = action
    return

def select_corse(obs,shipyard,turn,param,same_corse):
    Dir = ['ENWS','WSEN','ESWN','WNES']
    posi_x = shipyard.position.x
    posi_y = 20 - shipyard.position.y
    kore = obs['kore']
    kore = np.array(kore).reshape((21,21))
    kore = np.roll(kore,(10-posi_y,10-posi_x),axis = (0,1))
    best = [0,0,'ENWS',0]
#     print(turn)
    if not same_corse:
        for x in range(21):
            for y in range(21):
                if param.kore_his[y][x] > 0:
                    kore[y][x] *= 0.85

    for x in np.arange(1,6):
        for y in np.arange(1,6):
            kx = [10,10 - x,10,10 - x]
            ky = [10 - y,10,10,10 - y]
            for i,DIR in enumerate(Dir):
                score = kore[ky[i]:ky[i]+y+1,kx[i]:kx[i]+x+1].sum() - kore[ky[i]+1:ky[i]+y,kx[i]+1:kx[i]+x].sum()
                score /= 2*(x + y)
                if score > best[3]:
                    best[0] = x
                    best[1] = y
                    best[2] = Dir[i]
                    best[3] = score
#                     print(kore[ky[i]:ky[i]+y+1,kx[i]:kx[i]+x+1])
#                     print(x,y,Dir[i],score)
        
    return best[0],best[1],best[2]

def param_reset(param):
    param.box_num = [0]
    param.ship_num = [0]
    param.shipyard_count = 1
    param.max_shipyard = 1
    param.reserve = {'0-1':[None]*400}
    param.kore_his = np.array([[0]*21]*21)

    return

##-----------classes-------------

class param:
    box_num = [0]
    ship_num = [0]
    shipyard_count = 1
    max_shipyard = 1
    reserve = {'0-1':[None]*400}
    kore_his = np.array([[0]*21]*21)
    DIR = ['ENWS','WSEN','ESWN','WNES']

##-----------agents-------------

def bestdir_agent(obs, config):
    board = Board(obs, config)
    me=board.current_player
    turn = board.step
    if turn == 0: param_reset(param)
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    box_mining_bestdir(obs,config,me,turn,param)
    max_spawn(me,spawn_cost,kore_left,param)
    return me.next_actions

def bestdir_agent_plus(obs, config):
    board = Board(obs, config)
    me = board.current_player
    turn = board.step
    if turn == 0: param_reset(param)
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    box_mining_bestdir(obs,config,me,turn,param,same_dir = False)
    max_spawn(me,spawn_cost,kore_left,param)
    return me.next_actions
    
def bestcorse_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    bestcorse_minning(obs,config,me,turn,param)
    max_spawn(me,spawn_cost,kore_left,param)
    return me.next_actions

def bestcorse_agent_prus(obs, config):
    board = Board(obs, config)
    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    bestcorse_minning(obs,config,me,turn,param,same_corse = False)
    max_spawn(me,spawn_cost,kore_left,param)
    return me.next_actions

## 3-1 best dir agent

In [ ]:
from mining_useobs_test import bestdir_agent
step7 = env.run([bestdir_agent,do_nothing])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step7)

## 3-2 not_use_observation (比較用)

In [ ]:
step6_2 = env.run([size5_agent,do_nothing])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step6_2)

## 3-3 best dir agent vs size5 agent

In [ ]:
step_list = [step7,step6_2]
agent_name = ["use_observation","Don't_use"]
steps_compare(step_list,agent_name)

**霊夢:序盤の資源回収率は良いけど、後半の資源回収率は良くないね。**

**魔理沙:前の艦隊が通った航路は、資源が減っているはずだけど、その計算がされてないからじゃないかな。**

**Reimu: The resource recovery rate in the early stages is good, but the resource recovery rate in the latter half is not good.**

**Marisa: The route that the previous fleet took should be low in resources, but I think it's because it hasn't been calculated.**

## 3-4 best dir agent plus

**魔理沙:前の艦隊が通った航路だけ、0.9倍にして計算するようにしてみたぜ**

In [ ]:
from mining_useobs_test import bestdir_agent_plus
step8 = env.run([bestdir_agent_plus,do_nothing])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step8)

In [ ]:
step_list = [step8,step6_2]
agent_name = ["use_obs_ver2","Don't_use"]
steps_compare(step_list,agent_name)

## 3-5 best corse agent  

**observationを使って、期待値を出す**

**flightplanを作成**

**observationから、最も効率の良いコースを選択**

**造船を管理する関数**

In [ ]:
from mining_useobs_test import bestcorse_agent
step10 = env.run([bestcorse_agent,do_nothing])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step10)

**霊夢:このやり方だと、過去に選択したコースの影響が期待値に加味されてないね**

**魔理沙:一回選択したコースに一定の割合で割り算をしてあげないといけないね**

## 3-6 best corse agent plus

**フライトプランを作る関数**

**21×21のリストを作り、直近5ターンに決まった航路を記録し、その部分のKoreを15％減らして次の航路を計算させます。**

**回収効率良い航路を検索する関数**

**直近5ターンに決まった航路上のKoreは0.85倍しています。**

In [ ]:
from mining_useobs_test import bestcorse_agent_prus
step11 = env.run([bestcorse_agent_prus,do_nothing])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(step11)

In [ ]:
step_list = [step10,step11]
agent_name = ["not discount calc","discount calc"]
steps_compare(step_list,agent_name)

**過去に選択したコースの影響を加味させると、Allが30000を超えました。**

# V4 refactering

In [ ]:
%%writefile create_flight_course.py

def create_flight_course_square(ew,ns):
    s_dir='ESWN'
    p=[0,1,2,3]
    s_ew=str(ew-1) if ew>1 else str(-ew-1) if ew<-1 else '' 
    s_ns=str(ns-1) if ns>1 else str(-ns-1) if ns<-1 else '' 
    if ew<0: p[0],p[2]=p[2],p[0]
    if ns<0: p[1],p[3]=p[3],p[1]
        
    return s_dir[p[0]]+s_ew+s_dir[p[1]]+s_ns+s_dir[p[2]]+s_ew+s_dir[p[3]]

def create_flight_course_L(ew,ns):
    s_dir='ESWN'
    p=[0,1,2,3]
    s_ew=str(ew-1) if ew>1 else str(-ew-1) if ew<-1 else '' 
    if ew<0: p[0],p[2]=p[2],p[0]
    if ns<0: p[1],p[3]=p[3],p[1]
    
    return s_dir[p[0]]+s_ew+s_dir[p[1]]


def create_flight_course_box(ew,ns):
    res=[create_flight_course_square(ew,ns)]
    if ew>0:
        res=res+[create_flight_course_L(ew-x-1,ns) for x in range(ew-1) ]
    else:
        res=res+[create_flight_course_L(ew+x+1,ns) for x in range(-ew-1) ]
    return res    



In [ ]:
%%writefile make_ship_new.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
from kore_cfg import CFG

def spawn_all(agent):
    kore_left=agent.kore
    for shipyard in agent.shipyards:
        if not shipyard.next_action == None:
            continue
        n_ship=kore_left//CFG.spawn_cost
        if n_ship>shipyard.max_spawn:
            n_ship=shipyard.max_spawn
        if n_ship >0:
            shipyard.next_action = ShipyardAction.spawn_ships(n_ship)
            kore_left-=CFG.spawn_cost*n_ship
    return

def spawn(shipyard,remain_kore):
    n_ship=int(remain_kore//CFG.spawn_cost)
    if n_ship>shipyard.max_spawn:
        n_ship=shipyard.max_spawn
    if n_ship >0:
        shipyard.next_action = ShipyardAction.spawn_ships(n_ship)
        remain_kore-=CFG.spawn_cost*n_ship
    return

In [ ]:
%%writefile kore_cfg.py

class CFG:
    spawn_cost=10
    max_fleets=40

In [ ]:
%%writefile best_mining.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
import numpy as np
import re
from make_ship_new import spawn
from create_flight_course import * 
from kore_cfg import *

dir_ew=[1,-1,-1,1]
dir_ns=[1,1,-1,-1]                               

##-----------functions-------------
def box_mining(shipyard,turn,ew=5,ns=5):
    min_fleets=21+3*(abs(ew)-1)
    ship_count=shipyard.ship_count-reserve_ship(shipyard.id,turn)
    if ship_count >=min_fleets:
        course=create_flight_course_box(ew,ns)
        main_ship_count=max(21,ship_count//abs(ew))
        sub_ship_count=min((ship_count-main_ship_count)//(abs(ew)-1), main_ship_count-1)
        n_ships=[main_ship_count]+[sub_ship_count]*(abs(ew)-1)
        for i in range(abs(ew)):
            flight_plan[shipyard.id]['n_fleets'][turn+i*2]=n_ships[i] 
            flight_plan[shipyard.id]['course'][turn+i*2]=course[i] 
        shipyard.next_action=ShipyardAction.launch_fleet_with_flight_plan(n_ships[0], course[0])        
    return

def search_best_dir(kore_map,shipyard,box_size):
    global dir_ew,dir_ns
    kore = np.array(kore_map).reshape(21,21)
    kore = np.roll(kore,(shipyard.position.y-10,10-shipyard.position.x),axis = (0,1))
    
    best_dir=0
    best_score=0

    for i in range(4):
        pos_y1=10
        pos_x1=10
        pos_y2=10+dir_ns[i]*box_size
        pos_x2=10+dir_ew[i]*box_size
        if pos_x1>pos_x2: pos_x1,pos_x2=pos_x2,pos_x1
        if pos_y1>pos_y2: pos_y1,pos_y2=pos_y2,pos_y1
            
        score = kore[pos_y1:pos_y2+1,pos_x1:pos_x2+1].sum()

        if best_score < score:
            best_score=score
            best_dir=i
    return best_dir

def search_best_course(kore_map,shipyard,max_length=40):
    global dir_ew,dir_ns
    kore = np.array(kore_map).reshape(21,21)
    kore = np.roll(kore,(shipyard.position.y-10,10-shipyard.position.x),axis = (0,1))
    best_ew=0
    best_ns=0
    best_score=0

    for x in np.arange(1,11):
        for y in np.arange(1,11):
            for i in range(4):
                pos_y1=10
                pos_x1=10
                pos_y2=10+dir_ns[i]*y
                pos_x2=10+dir_ew[i]*x
                if pos_x1>pos_x2: pos_x1,pos_x2=pos_x2,pos_x1
                if pos_y1>pos_y2: pos_y1,pos_y2=pos_y2,pos_y1
                score = kore[pos_y1:pos_y2+1,pos_x1:pos_x2+1].sum()-kore[pos_y1+1:pos_y2,pos_x1+1:pos_x2].sum()
                score /= 2*(x + y)
                if score > best_score and max_length>=2*(x+y):
                    best_score=score
                    best_ew=dir_ew[i]*x
                    best_ns=dir_ns[i]*y
        
    return best_score,best_ew,best_ns


flight_plan={}
def _create_flight_plan(shipyard_id):
    global flight_plan
    flight_plan[shipyard_id]={}
    flight_plan[shipyard_id]['n_fleets']=[0]*480
    flight_plan[shipyard_id]['course']=[""]*480
    return

flight_map=[0]*441
def _create_kore_map(obs_kore):
    global flight_map
    np_obs=np.array(obs_kore).astype(float)
    np_fm=np.array(flight_map).astype(float)
    np_gain=np.ones(441).astype(float)
    np_gain[np.where(np_fm>0)[0]]=0.9
    np_obs=np_obs*np_gain
    return list(np_obs)

def _update_flight_map():
    global flight_map
    np_fm=np.array(flight_map).astype(float)
    np_fm-=np.ones(441).astype(float)
    np_fm[np.where(np_fm<0)[0]]=0
    flight_map=list(np_fm)
    return

def _add_flight_map(shipyard_pos,ew,ns,square=True):
    global flight_map
    a_map=np.zeros(441).reshape(21,21)
    pos_y1=10
    pos_x1=10
    pos_y2=10-ew
    pos_x2=10+ns
    if pos_x1>pos_x2: pos_x1,pos_x2=pos_x2,pos_x1
    if pos_y1>pos_y2: pos_y1,pos_y2=pos_y2,pos_y1           
    a_map[pos_y1:pos_y2+1,pos_x1:pos_x2+1]=(abs(ew)+abs(ns))*2
    if square: a_map[pos_y1+1:pos_y2,pos_x1+1:pos_x2]=0    
    a_map = np.roll(a_map,(10-shipyard_pos.y,shipyard_pos.x-10),axis = (0,1))
    a_map=list(a_map.reshape(441))
    flight_map=[a_map[i] if a_map[i]>flight_map[i] else flight_map[i] for i in range(441)]

def _init():
    global flight_map
    global flight_plan
    flight_plan={}
    flight_map=[0]*441

def reserve_ship(shipyard_id,turn):
    return sum(flight_plan[shipyard_id]['n_fleets'][turn:])

def bestdir_agent(obs, config):
    global flight_map
    global flight_plan
    global dir_ew,dir_ns

    board = Board(obs, config)
    me=board.current_player
    turn = board.step
    if turn == 0: _init()
    box_size=5
    remain_kore=me.kore
    _update_flight_map()
    kore_map=_create_kore_map(obs['kore'])
    
    for shipyard in me.shipyards:
        if shipyard.id not in flight_plan:
            _create_flight_plan(shipyard.id)
        n_fleets=flight_plan[shipyard.id]['n_fleets'][turn]
        if n_fleets>0:
            flight_course=flight_plan[shipyard.id]['course'][turn]
            shipyard.next_action=ShipyardAction.launch_fleet_with_flight_plan(n_fleets, flight_course)
        else:
            t_side=search_best_dir(kore_map,shipyard,box_size)
            box_mining(shipyard,turn,ew=dir_ew[t_side]*box_size,ns=dir_ns[t_side]*box_size)
            if not shipyard.next_action == None:
                _add_flight_map(shipyard.position,dir_ew[t_side]*box_size,dir_ns[t_side]*box_size,square=False)
        if shipyard.next_action==None: spawn(shipyard,remain_kore)

    return me.next_actions
    
def bestcourse_agent(obs, config):
    global flight_map

    board = Board(obs, config)
    me=board.current_player
    turn = board.step
    if turn == 0: _init()
    remain_kore=me.kore
    _update_flight_map()
    kore_map=_create_kore_map(obs['kore'])
                                
    for shipyard in me.shipyards:
        if shipyard.ship_count>=21:
            score,ew,ns=search_best_course(kore_map,shipyard)
            n_fleets=min(CFG.max_fleets,shipyard.ship_count)
            flight_course=create_flight_course_square(ew,ns)
            shipyard.next_action=ShipyardAction.launch_fleet_with_flight_plan(int(n_fleets), flight_course)
            _add_flight_map(shipyard.position,ew,ns)
        else:
            spawn(shipyard,remain_kore)
    return me.next_actions


In [ ]:
from kaggle_environments import make
env = make("kore_fleets",configuration={"randomSeed":42}, debug=True)
from best_mining import bestcourse_agent
steps = env.run([bestcourse_agent,do_nothing])
env.render(mode="ipython", width=800, height=640)

In [ ]:
steps_analyze(steps)

In [ ]:
from mining import bestdir_agent
steps = env.run([bestdir_agent,bestcourse_agent])
env.render(mode="ipython", width=800, height=640)